# test a model

Test youy rmodel right after training in a kubeflow pipeline, or run this function independently.  In addition, the plotting components in **[test_classifier.py]()** can also be run independently.

In [57]:
import mlrun
import os
import numpy as np
mlrun.mlconf.dbpath = 'http://mlrun-api:8080'

## parameters

In [58]:
CODE_BASE          = '/User/repos/functions'

MODEL_FILE         = '/User/mlrun/models/lgb-classifier.pkl'

TARGET_DATA_PATH   = '/User/mlrun/splitter'
XTEST_FILE         = '/User/mlrun/splitter/xtest.pqt'
YTEST_FILE         = '/User/mlrun/splitter/ytest.pqt'

## run tests

In [122]:
yaml_name = os.path.join(CODE_BASE, 'evaluation', 'test-classifier.yaml')
if not os.path.isfile(yaml_name):
    testfn = mlrun.code_to_function(
        kind='job', 
        image='yjbds/mlrun-ds:latest',
        filename=os.path.join(CODE_BASE, 'evaluation', 'test-classifier.py'))

    testfn.export(os.path.join(CODE_BASE, 'evaluation', 'test-classifier.yaml'))

[mlrun] 2020-01-26 00:40:51,865 function spec saved to path: /User/repos/functions/evaluation/test-classifier.yaml


In [123]:
testfn = mlrun.import_function(
    os.path.join(CODE_BASE, 'evaluation', 'test-classifier.yaml')
).apply(mlrun.mount_v3io())

In [124]:
testfn.deploy(skip_deployed=True, with_mlrun=False)

'ready'

In [125]:
task = mlrun.NewTask()
task.with_params(
    model=MODEL_FILE,
    xtest=XTEST_FILE,
    ytest=YTEST_FILE,
    target_path=TARGET_DATA_PATH)

In [126]:
tsk_run = testfn.run(task, handler='test_model')

[mlrun] 2020-01-26 00:41:47,446 starting run test_model uid=ea859673d08b4eccbd7746b9d36fb8e8  -> http://mlrun-api:8080
[mlrun] 2020-01-26 00:41:47,550 Job is running in the background, pod: test-model-shb4b
[mlrun] 2020-01-26 00:42:01,686 log artifact roc.html at roc.html, size: 40483, db: Y
[mlrun] 2020-01-26 00:42:02,895 log artifact confusion_matrix.html at confusion_matrix.html, size: 15292, db: Y
[mlrun] 2020-01-26 00:42:03,498 log artifact feature-importances-plot.html at feature-importances-plot.html, size: 67516, db: Y
[mlrun] 2020-01-26 00:42:03,525 log artifact feature-importances-table at /User/mlrun/splitter/feature-importances-table.csv, size: None, db: Y

[mlrun] 2020-01-26 00:42:03,596 run executed, status=completed
/opt/conda/lib/python3.7/site-packages/pyarrow/pandas_compat.py:708: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels = getattr(columns, 'labels', None) or [
/opt/conda/lib/python3.7/site-packages/pyarrow/pandas_compat.py:

uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...6fb8e8,0,Jan 26 00:41:54,completed,test-classifier,host=test-model-shb4bkind=jobowner=admin,,model=/User/mlrun/models/lgb-classifier.pkltarget_path=/User/mlrun/splitterxtest=/User/mlrun/splitter/xtest.pqtytest=/User/mlrun/splitter/ytest.pqt,,roc.htmlconfusion_matrix.htmlfeature-importances-plot.htmlfeature-importances-table


to track results use .show() or .logs() or in CLI: 
!mlrun get run ea859673d08b4eccbd7746b9d36fb8e8  , !mlrun logs ea859673d08b4eccbd7746b9d36fb8e8 
[mlrun] 2020-01-26 00:42:06,762 run executed, status=completed
